
# CS667 Project 2 – Regression on Retail Sales (End-to-End)
This notebook covers: data prep, feature engineering, train/test split, model training (Linear, Tree, RandomForest, GradientBoosting), evaluation (R², MAE, MSE, RMSE), and plots (correlation heatmap, actual vs predicted, residuals, feature importances).


In [ ]:

# Setup
import os, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn import metrics

DATA_PATH = "/mnt/data/retail_data_synthetic_50k.xlsx"
df = pd.read_excel(DATA_PATH)
df.head()


In [ ]:

# Normalize columns and detect target
df.columns = [c.strip().replace("\n"," ").replace("\r"," ") for c in df.columns]

possible_targets = ["Total Amount","Total_Amount","total_amount","TOTAL_AMOUNT","Revenue","Sales","Total","Amount"]
target_col = None
for cand in possible_targets:
    if cand in df.columns:
        target_col = cand
        break
if target_col is None:
    candidates = df.select_dtypes(include=[np.number]).columns.tolist()
    target_col = candidates[-1]
print("Target column:", target_col)

# Date features (if available)
date_cols = [c for c in df.columns if "date" in c.lower()]
if date_cols:
    dcol = date_cols[0]
    df[dcol] = pd.to_datetime(df[dcol], errors="coerce")
    df["year"] = df[dcol].dt.year
    df["month"] = df[dcol].dt.month
    df["dayofweek"] = df[dcol].dt.dayofweek
    df["is_weekend"] = df["dayofweek"].isin([5,6]).astype(int)

    def month_to_season(m):
        if pd.isna(m):
            return np.nan
        m = int(m)
        if m in [12,1,2]: return "winter"
        if m in [3,4,5]:  return "spring"
        if m in [6,7,8]:  return "summer"
        return "fall"
    df["season"] = df["month"].apply(month_to_season)

# Feature selection
id_like = set(["Transaction ID","Transaction_ID","transaction_id","Customer ID","Customer_ID","customer_id"])
feature_cols = [c for c in df.columns if c != target_col and c not in id_like]

numeric_cols = df[feature_cols].select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = [c for c in feature_cols if c not in numeric_cols]

numeric_transformer = Pipeline(steps=[("imputer", SimpleImputer(strategy="median")),
                                      ("scaler", StandardScaler())])
categorical_transformer = Pipeline(steps=[("imputer", SimpleImputer(strategy="most_frequent")),
                                          ("encoder", OneHotEncoder(handle_unknown="ignore"))])

preprocess = ColumnTransformer(
    transformers=[("num", numeric_transformer, numeric_cols),
                  ("cat", categorical_transformer, categorical_cols)]
)

X = df[feature_cols].copy()
y = df[target_col].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Train/Test shapes:", X_train.shape, X_test.shape)


In [ ]:

# Train and evaluate models
candidates = {
    "LinearRegression": LinearRegression(),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    "GradientBoosting": GradientBoostingRegressor(random_state=42)
}

def evaluate_model(name, model):
    pipe = Pipeline(steps=[("preprocess", preprocess), ("model", model)])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    r2 = metrics.r2_score(y_test, preds)
    mae = metrics.mean_absolute_error(y_test, preds)
    mse = metrics.mean_squared_error(y_test, preds)
    rmse = math.sqrt(mse)
    return name, pipe, r2, mae, mse, rmse, preds

records = []
for name, model in candidates.items():
    try:
        rec = evaluate_model(name, model)
        records.append(rec)
    except Exception as e:
        print("[Warn] Skipping", name, "->", e)

perf = pd.DataFrame([{"Model": r[0], "R2": r[2], "MAE": r[3], "MSE": r[4], "RMSE": r[5]} for r in records])
perf.sort_values(["R2","RMSE"], ascending=[False, True])


In [ ]:

# Pick best model
records_sorted = sorted(records, key=lambda r: (r[2], -r[5]), reverse=True)
best = records_sorted[0]
best_name, best_pipe, r2, mae, mse, rmse, preds = best
print("Best model:", best_name, "| R2:", r2, "| RMSE:", rmse)
perf


In [ ]:

# Correlation heatmap
num_for_corr = df.select_dtypes(include=[np.number])
if num_for_corr.shape[1] >= 2:
    corr = num_for_corr.corr(numeric_only=True)
    plt.figure()
    plt.imshow(corr.values, interpolation="nearest")
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.index)), corr.index)
    plt.title("Correlation Heatmap (Numeric Features)")
    plt.colorbar()
    plt.tight_layout()
    plt.show()


In [ ]:

# Actual vs Predicted
plt.figure()
plt.scatter(y_test, preds, alpha=0.5)
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title(f"Actual vs Predicted ({best_name})")
plt.tight_layout()
plt.show()


In [ ]:

# Residuals plot
residuals = y_test - preds
plt.figure()
plt.scatter(preds, residuals, alpha=0.5)
plt.axhline(0, linestyle="--")
plt.xlabel("Predicted")
plt.ylabel("Residuals (Actual - Predicted)")
plt.title(f"Residual Plot ({best_name})")
plt.tight_layout()
plt.show()


In [ ]:

# Feature importances / coefficients
try:
    ohe = best_pipe.named_steps["preprocess"].named_transformers_["cat"].named_steps["encoder"] if "cat" in best_pipe.named_steps["preprocess"].transformers_[1][0] else None
except Exception:
    try:
        ohe = best_pipe.named_steps["preprocess"].named_transformers_["cat"].named_steps["encoder"]
    except Exception:
        ohe = None

if ohe is None:
    try:
        ohe = best_pipe.named_steps["preprocess"].named_transformers_["cat"].named_steps["encoder"]
    except Exception:
        ohe = None

num_features = numeric_cols
cat_features = list(ohe.get_feature_names_out(categorical_cols)) if ohe is not None else []
all_features = list(num_features) + list(cat_features)

model_obj = best_pipe.named_steps["model"]
importances = None
if hasattr(model_obj, "feature_importances_"):
    importances = model_obj.feature_importances_
elif hasattr(model_obj, "coef_"):
    coef = model_obj.coef_
    importances = coef if np.ndim(coef) == 1 else coef.ravel()

if importances is not None and len(all_features) == len(importances):
    fi = pd.DataFrame({ "feature": all_features, "importance": np.abs(importances) }).sort_values("importance", ascending=False).head(25)
    plt.figure(figsize=(8, max(4, len(fi)*0.3)))
    plt.barh(fi["feature"], fi["importance"])
    plt.gca().invert_yaxis()
    plt.title(f"Top Feature Importances ({best_name})")
    plt.xlabel("Importance (abs)")
    plt.tight_layout()
    plt.show()
    fi.head(25)
else:
    print("Feature importances/coefficients not available or length mismatch.")
